In [3]:
import requests
import json
import io
import pandas as pd

sql0 = "SELECT COUNT(*) FROM pullRequestMergedEvents"
sql1 = "select organization, sum(count) as pr_count from pullRequestMergedEvents group by organization order by pr_count desc"
sql2 = "select userId, sum(count) as pr_count from pullRequestMergedEvents group by userId order by pr_count desc"
# Accessing inverted column organization
sql3 = "select * FROM pullRequestMergedEvents where organization = 'itsprivate'"
sql4 = "select * FROM pullRequestMergedEvents where repo = 'leitos'"
sql5 = "select * FROM pullRequestMergedEvents where userId = 'rdmoedas'"

data = {
    "sql" : sql4
}

response = requests.post('http://pinot-broker.pinot:8099/query/sql', json=data)
response_json=response.json()
print(response_json)

{'resultTable': {'dataSchema': {'columnDataTypes': ['STRING'], 'columnNames': ['*']}, 'rows': []}, 'exceptions': [], 'numServersQueried': 1, 'numServersResponded': 1, 'numSegmentsQueried': 1, 'numSegmentsProcessed': 0, 'numSegmentsMatched': 0, 'numConsumingSegmentsQueried': 1, 'numDocsScanned': 0, 'numEntriesScannedInFilter': 0, 'numEntriesScannedPostFilter': 0, 'numGroupsLimitReached': False, 'totalDocs': 0, 'timeUsedMs': 214, 'segmentStatistics': [], 'traceInfo': {}, 'minConsumingFreshnessTimeMs': 9223372036854775807}


## Create a Pinot Schema

In [32]:
schema_config = {
  "schemaName": "pullRequestMergedEvents",
  "dimensionFieldSpecs": [
    {
      "name": "title",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "labels",
      "dataType": "STRING",
      "singleValueField": False,
      "defaultNullValue": ""
    },
    {
      "name": "userId",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "userType",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "authorAssociation",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "mergedBy",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "assignees",
      "dataType": "STRING",
      "singleValueField": False,
      "defaultNullValue": ""
    },
    {
      "name": "authors",
      "dataType": "STRING",
      "singleValueField": False,
      "defaultNullValue": ""
    },
    {
      "name": "committers",
      "dataType": "STRING",
      "singleValueField": False,
      "defaultNullValue": ""
    },
    {
      "name": "requestedReviewers",
      "dataType": "STRING",
      "singleValueField": False,
      "defaultNullValue": ""
    },
    {
      "name": "requestedTeams",
      "dataType": "STRING",
      "singleValueField": False,
      "defaultNullValue": ""
    },
    {
      "name": "reviewers",
      "dataType": "STRING",
      "singleValueField": False,
      "defaultNullValue": ""
    },
    {
      "name": "commenters",
      "dataType": "STRING",
      "singleValueField": False,
      "defaultNullValue": ""
    },
    {
      "name": "repo",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "organization",
      "dataType": "STRING",
      "defaultNullValue": ""
    }
  ],
  "metricFieldSpecs": [
    {
      "name": "count",
      "dataType": "LONG",
      "defaultNullValue": 1
    },
    {
      "name": "numComments",
      "dataType": "LONG"
    },
    {
      "name": "numReviewComments",
      "dataType": "LONG"
    },
    {
      "name": "numCommits",
      "dataType": "LONG"
    },
    {
      "name": "numLinesAdded",
      "dataType": "LONG"
    },
    {
      "name": "numLinesDeleted",
      "dataType": "LONG"
    },
    {
      "name": "numFilesChanged",
      "dataType": "LONG"
    },
    {
      "name": "numAuthors",
      "dataType": "LONG"
    },
    {
      "name": "numCommitters",
      "dataType": "LONG"
    },
    {
      "name": "numReviewers",
      "dataType": "LONG"
    },
    {
      "name": "numCommenters",
      "dataType": "LONG"
    },
    {
      "name": "createdTimeMillis",
      "dataType": "LONG"
    },
    {
      "name": "elapsedTimeMillis",
      "dataType": "LONG"
    }
  ],
  "timeFieldSpec": {
    "incomingGranularitySpec": {
      "timeType": "MILLISECONDS",
      "timeFormat": "EPOCH",
      "dataType": "LONG",
      "name": "mergedTimeMillis"
    }
  }
}

response = requests.post('http://pinot-controller.pinot:9000/schemas?override=false', json=schema_config)
print(response.json())

{'status': 'pullRequestMergedEvents2 successfully added'}


In [29]:
response = requests.get('http://pinot-controller.pinot:9000/schemas')
print(response.json())

['pullRequestMergedEvents', 'pullRequestMergedEvents2']


In [30]:
response = requests.delete('http://pinot-controller.pinot:9000/schemas/pullRequestMergedEvents')
print(response.json())

{'status': 'Schema pullRequestMergedEvents2 deleted'}


In [31]:
response = requests.get('http://pinot-controller.pinot:9000/schemas')
print(response.json())

['pullRequestMergedEvents']


## Create Kafka topic

In [33]:
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers="pinot-kafka.pinot:9092", 
    client_id='test'
)

topic_list = []
topic_list.append(NewTopic(name="pullRequestMergedEvents", num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='pullRequestMergedEvents2', error_code=0, error_message=None)])

In [34]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(group_id='test', bootstrap_servers=['pinot-kafka.pinot:9092'])
consumer.topics()

{'pullRequestMergedEvents', 'pullRequestMergedEvents2'}

In [37]:
admin_client.delete_topics(topics=["pullRequestMergedEvents"])

UnknownError: [Error -1] UnknownError: Request 'DeleteTopicsRequest_v3(topics=['pullRequestMergedEvents2'], timeout=30000)' failed with response 'DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='pullRequestMergedEvents2', error_code=73)])'.

## Create Pinot table

In [45]:
table_config = {
  "tableName": "pullRequestMergedEvents",
  "tableType": "REALTIME",
  "segmentsConfig": {
    "timeColumnName": "mergedTimeMillis",
    "timeType": "MILLISECONDS",
    "retentionTimeUnit": "DAYS",
    "retentionTimeValue": "60",
    "schemaName": "pullRequestMergedEvents",
    "replication": "1",
    "replicasPerPartition": "1"
  },
  "tenants": {},
  "tableIndexConfig": {
    "loadMode": "MMAP",
    "invertedIndexColumns": [
      "organization",
      "repo"
    ],
    "streamConfigs": {
      "streamType": "kafka",
      "stream.kafka.consumer.type": "simple",
      "stream.kafka.topic.name": "pullRequestMergedEvents",
      "stream.kafka.decoder.class.name": "org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder",
      "stream.kafka.consumer.factory.class.name": "org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory",
      "stream.kafka.zk.broker.url": "pinot-kafka-zookeeper:2181",
      "stream.kafka.broker.list": "pinot-kafka:9092",
      "realtime.segment.flush.threshold.time": "12h",
      "realtime.segment.flush.threshold.size": "100000",
      "stream.kafka.consumer.prop.auto.offset.reset": "smallest"
    }
  },
  "metadata": {
    "customConfigs": {}
  }
}

response = requests.post('http://pinot-controller.pinot:9000/tables', json=table_config)
print(response.json())

{'status': 'Table pullRequestMergedEvents_REALTIME succesfully added'}


In [41]:
response = requests.get('http://pinot-controller.pinot:9000/tables')
print(response.json())

{'tables': ['pullRequestMergedEvents']}


In [42]:
response = requests.get('http://pinot-controller.pinot:9000/tables/pullRequestMergedEvents')
print(response.json())

{'REALTIME': {'tableName': 'pullRequestMergedEvents_REALTIME', 'tableType': 'REALTIME', 'segmentsConfig': {'timeColumnName': 'mergedTimeMillis', 'retentionTimeUnit': 'DAYS', 'retentionTimeValue': '60', 'timeType': 'MILLISECONDS', 'schemaName': 'pullRequestMergedEvents', 'replication': '1', 'replicasPerPartition': '1'}, 'tenants': {'broker': 'DefaultTenant', 'server': 'DefaultTenant'}, 'tableIndexConfig': {'loadMode': 'MMAP', 'streamConfigs': {'streamType': 'kafka', 'stream.kafka.consumer.type': 'simple', 'stream.kafka.topic.name': 'pullRequestMergedEvents', 'stream.kafka.decoder.class.name': 'org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder', 'stream.kafka.consumer.factory.class.name': 'org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory', 'stream.kafka.zk.broker.url': 'pinot-kafka-zookeeper:2181', 'stream.kafka.broker.list': 'pinot-kafka:9092', 'realtime.segment.flush.threshold.time': '12h', 'realtime.segment.flush.threshold.size': '100000', 'stream.kafka.consumer.

In [43]:
response = requests.delete('http://pinot-controller.pinot:9000/tables/pullRequestMergedEvents')
print(response.json())

{'status': 'Tables: [pullRequestMergedEvents_REALTIME] deleted'}


In [46]:
response = requests.get('http://pinot-controller.pinot:9000/tables')
print(response.json())

{'tables': ['pullRequestMergedEvents']}


## Write into Kafka topic

In [50]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers=['pinot-kafka.pinot:9092'], value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [51]:
row = {
  "assignees": [
    ""
  ],
  "authorAssociation": "OWNER",
  "authors": [
    "chamod-gamage"
  ],
  "commenters": [
    ""
  ],
  "committers": [
    "chamod-gamage"
  ],
  "count": 1,
  "createdTimeMillis": 1616542979000,
  "elapsedTimeMillis": 7000,
  "labels": [
    ""
  ],
  "mergedBy": "chamod-gamage",
  "mergedTimeMillis": 1616542986000,
  "numAuthors": 1,
  "numCommenters": 0,
  "numComments": 0,
  "numCommits": 1,
  "numCommitters": 1,
  "numFilesChanged": 3,
  "numLinesAdded": 23192,
  "numLinesDeleted": 12666,
  "numReviewComments": 0,
  "numReviewers": 0,
  "organization": "chamod-gamage",
  "repo": "rideshare-game",
  "requestedReviewers": [
    ""
  ],
  "requestedTeams": [
    ""
  ],
  "reviewers": [
    ""
  ],
  "title": "remove babel-loader from package.json",
  "userId": "chamod-gamage",
  "userType": "User"
}

for e in range(100):
    producer.send('pullRequestMergedEvents', value=row)

## Read inserted data from Pinot

In [71]:
data = {
    "sql" : "select * from pullRequestMergedEvents limit 10"
}

response = requests.post('http://pinot-broker.pinot:8099/query/sql', json=data)
response_json=response.json()
print(response_json)

{'resultTable': {'dataSchema': {'columnDataTypes': ['STRING_ARRAY', 'STRING', 'STRING_ARRAY', 'STRING_ARRAY', 'STRING_ARRAY', 'LONG', 'LONG', 'LONG', 'STRING_ARRAY', 'STRING', 'LONG', 'LONG', 'LONG', 'LONG', 'LONG', 'LONG', 'LONG', 'LONG', 'LONG', 'LONG', 'LONG', 'STRING', 'STRING', 'STRING_ARRAY', 'STRING_ARRAY', 'STRING_ARRAY', 'STRING', 'STRING', 'STRING'], 'columnNames': ['assignees', 'authorAssociation', 'authors', 'commenters', 'committers', 'count', 'createdTimeMillis', 'elapsedTimeMillis', 'labels', 'mergedBy', 'mergedTimeMillis', 'numAuthors', 'numCommenters', 'numComments', 'numCommits', 'numCommitters', 'numFilesChanged', 'numLinesAdded', 'numLinesDeleted', 'numReviewComments', 'numReviewers', 'organization', 'repo', 'requestedReviewers', 'requestedTeams', 'reviewers', 'title', 'userId', 'userType']}, 'rows': [[[''], 'OWNER', ['chamod-gamage'], [''], ['chamod-gamage'], 1, 1616542979000, 7000, [''], 'chamod-gamage', 1616542986000, 1, 0, 0, 1, 1, 3, 23192, 12666, 0, 0, 'chamod

In [70]:
import pandas

columnNames = response_json['resultTable']['dataSchema']['columnNames']
rows = response_json['resultTable']['rows']

dataframe = pandas.DataFrame(columns=columnNames,data=rows)
dataframe

,assignees,authorAssociation,authors,commenters,committers,count,createdTimeMillis,elapsedTimeMillis,labels,mergedBy,...,numReviewComments,numReviewers,organization,repo,requestedReviewers,requestedTeams,reviewers,title,userId,userType
0,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
1,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
2,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
3,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
4,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
5,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
6,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
7,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
8,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User
9,[],OWNER,[chamod-gamage],[],[chamod-gamage],1,1616542979000,7000,[],chamod-gamage,...,0,0,chamod-gamage,rideshare-game,[],[],[],remove babel-loader from package.json,chamod-gamage,User


## Load Trips example

In [98]:
response = requests.post('http://pinot-controller.pinot:9000/schemas?override=true', json=json.load(open('trips/trips_schema.json')))
print(response)
print(response.text)

<Response [200]>
{"status":"trips successfully added"}


In [108]:
response = requests.post('http://pinot-controller.pinot:9000/tables', json=json.load(open('trips/trips_realtime_table_config.json')))
print(response)
print(response.text)

<Response [200]>
{"status":"Table trips_REALTIME succesfully added"}


In [109]:
admin_client.create_topics(new_topics=[NewTopic(name="trips", num_partitions=1, replication_factor=1)], validate_only=False)

TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='trips', num_partitions=1, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='trips', error_code=36, error_message="Topic 'trips' already exists.")])'.

In [110]:
example_trips = json.load(open('trips/example_trips.json'))

for trip in example_trips:
    producer.send('trips', value=trip)

In [111]:
response = requests.post('http://pinot-broker.pinot:8099/query/sql', json={
    "sql" : "select * from trips limit 10"
})
response_json=response.json()

columnNames = response_json['resultTable']['dataSchema']['columnNames']
rows = response_json['resultTable']['rows']

dataframe = pandas.DataFrame(columns=columnNames,data=rows)
dataframe

,count,driver_name,driver_rating,end_location,end_zip_code,license_plate,payment_amount,payment_tip_amount,request_time_millis,rider_is_premium,rider_name,rider_rating,start_location,start_zip_code,trip_end_time_millis,trip_start_time_millis,trip_wait_time_millis
0,1,,0,,,,0.0,0.0,-9223372036854775808,0,,0,,,-9223372036854775808,-9223372036854775808,0
1,1,Tim,5,Petersberg,36100,FD-TE2911,0.0,0.0,1616504629000,1,Nikola,5,Schlüchtern,36381,1616904629000,1616704629000,60000
2,1,Tim,5,Petersberg,36100,FD-TE2911,0.0,0.0,1616504629000,1,Nikola,5,Schlüchtern,36381,1616904629000,1616704629000,60000
3,1,Tim,5,Petersberg,36100,FD-TE2911,0.0,0.0,1616504629000,1,Nikola,5,Schlüchtern,36381,1616904629000,1616704629000,60000
4,1,Tim,5,Petersberg,36100,FD-TE2911,0.0,0.0,1616504629000,1,Nikola,5,Schlüchtern,36381,1616904629000,1616704629000,60000
